In [0]:
!rm -f ru_tg_train.tar.gz
!wget https://www.dropbox.com/s/1ecl9orr2tagcgi/ru_tg_train.tar.gz
!rm -f ru_tg_train.json
!tar -xzvf ru_tg_train.tar.gz
!rm ru_tg_train.tar.gz

--2020-01-08 13:24:58--  https://www.dropbox.com/s/1ecl9orr2tagcgi/ru_tg_train.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/1ecl9orr2tagcgi/ru_tg_train.tar.gz [following]
--2020-01-08 13:24:58--  https://www.dropbox.com/s/raw/1ecl9orr2tagcgi/ru_tg_train.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca509f2d4dd0d1a3e01910d2bc9.dl.dropboxusercontent.com/cd/0/inline/AvwCABRnDGDgz8golAjaCf8hsDJ9gcsnqkRlxlJ-1IaVc6omxCjzSSUXgH8XRy0Mxjf6Jf3nhfOfDbFWzwg0KTiA_3XlVKOpmtH_EC53f5CfUR1jp-0WlNCJ1-JvbOK2VLw/file# [following]
--2020-01-08 13:24:58--  https://uca509f2d4dd0d1a3e01910d2bc9.dl.dropboxusercontent.com/cd/0/inline/AvwCABRnDGDgz8golAjaCf8hsDJ9gcsnqkRlxlJ-1IaVc6omxCjzSSUXgH8XRy0Mxjf6Jf3nhfOfDbFWzwg0KTiA_3XlVKOp

In [0]:
!rm -f en_tg_train.tar.gz
!wget https://www.dropbox.com/s/umd8tyx4wz1wquq/en_tg_train.tar.gz
!rm -f en_tg_train.json
!tar -xzvf en_tg_train.tar.gz
!rm en_tg_train.tar.gz

--2020-01-08 13:25:20--  https://www.dropbox.com/s/umd8tyx4wz1wquq/en_tg_train.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/umd8tyx4wz1wquq/en_tg_train.tar.gz [following]
--2020-01-08 13:25:20--  https://www.dropbox.com/s/raw/umd8tyx4wz1wquq/en_tg_train.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc24c2fd75be4eb957ecccdfad7c.dl.dropboxusercontent.com/cd/0/inline/AvwAbhCQ7erV7XVbcojfnnOQFYtNxM_3wxBCVIUvGsOsp4g6JwOb0MG8rLtPeEIA6s4vulf-tufOZ_57j0jEZ-_NrjkZoNIKHBhBzNSlNp2iA3CIs1FTqbkC1dRbxjvEdaU/file# [following]
--2020-01-08 13:25:21--  https://uc24c2fd75be4eb957ecccdfad7c.dl.dropboxusercontent.com/cd/0/inline/AvwAbhCQ7erV7XVbcojfnnOQFYtNxM_3wxBCVIUvGsOsp4g6JwOb0MG8rLtPeEIA6s4vulf-tufOZ_57j0jEZ-_NrjkZoNIK

In [0]:
!pip install url-normalize

In [0]:
import sys
import json
from url_normalize import url_normalize
from urllib.parse import urlparse   


def normalize_url(url):
    url_n = url_normalize(url)
    parsed = urlparse(url_n)
    scheme = "%s://" % parsed.scheme
    res = parsed.geturl().replace(scheme, '', 1)
    host = parsed.netloc
    if res.startswith('www.'):
        res = res.replace('www.', '')
        host = host.replace('www.', '')
    return res, host

data_files = [json.load(open("en_tg_train.json")), json.load(open("ru_tg_train.json"))]

url_set = set()
host_set = set()
for data in data_files:
    for row in data:
        url = row["url"]
        url_n, url_host = normalize_url(url)
        url_set.add(url_n)
        host_set.add(url_host)

In [0]:
from collections import Counter, defaultdict

count = Counter()
host_to_host = defaultdict(Counter)
for data in data_files:
    for row in data:
        url = row["url"]
        url_n, url_host = normalize_url(url)
        if "out_links" not in row:
            continue
        for link in row["out_links"]:
            link_n, link_host = normalize_url(link)
            if url_host != link_host and link_n in url_set:
                count[link] += 1
            if url_host != link_host and link_host in host_set:
                host_to_host[url_host][link_host] += 1

h2i = {}
i2h = []
for host in host_to_host:
    h2i[host] = len(i2h)
    i2h.append(host)

In [0]:
import numpy as np

size = len(i2h)
E = np.ones((size, size))
E = E / np.sum(E, axis=1, keepdims=True)
print(E)

[[0.00121507 0.00121507 0.00121507 ... 0.00121507 0.00121507 0.00121507]
 [0.00121507 0.00121507 0.00121507 ... 0.00121507 0.00121507 0.00121507]
 [0.00121507 0.00121507 0.00121507 ... 0.00121507 0.00121507 0.00121507]
 ...
 [0.00121507 0.00121507 0.00121507 ... 0.00121507 0.00121507 0.00121507]
 [0.00121507 0.00121507 0.00121507 ... 0.00121507 0.00121507 0.00121507]
 [0.00121507 0.00121507 0.00121507 ... 0.00121507 0.00121507 0.00121507]]


In [0]:
matrix = np.zeros((size, size))

for host in host_to_host:
    for link_host in host_to_host[host]:
        if link_host not in h2i:
            continue
        matrix[h2i[host]][h2i[link_host]] = host_to_host[host][link_host]
print(matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
for i in range(size):
    if np.sum(matrix[i]) == 0:
        matrix[i] = np.ones(size)
print(matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
matrix = matrix / np.sum(matrix, axis=1, keepdims=True)
matrix = 0.7 * matrix + 0.3 * E
print(matrix)

[[0.00036452 0.00036452 0.00036452 ... 0.00036452 0.00036452 0.00036452]
 [0.00216401 0.00036452 0.00036452 ... 0.00036452 0.00036452 0.00036452]
 [0.00036452 0.00036452 0.00036452 ... 0.00036452 0.00036452 0.00036452]
 ...
 [0.00036452 0.00036452 0.00036452 ... 0.00036452 0.00036452 0.00036452]
 [0.00036452 0.00036452 0.00036452 ... 0.00036452 0.00036452 0.00036452]
 [0.00036452 0.00036452 0.00036452 ... 0.00036452 0.00036452 0.00036452]]


In [0]:
x = np.ones((1, size))
x = x / np.sum(x, axis=1, keepdims=True)
print(x)

[[0.00121507 0.00121507 0.00121507 0.00121507 0.00121507 0.00121507
  0.00121507 0.00121507 0.00121507 0.00121507 0.00121507 0.00121507
  0.00121507 0.00121507 0.00121507 0.00121507 0.00121507 0.00121507
  0.00121507 0.00121507 0.00121507 0.00121507 0.00121507 0.00121507
  0.00121507 0.00121507 0.00121507 0.00121507 0.00121507 0.00121507
  0.00121507 0.00121507 0.00121507 0.00121507 0.00121507 0.00121507
  0.00121507 0.00121507 0.00121507 0.00121507 0.00121507 0.00121507
  0.00121507 0.00121507 0.00121507 0.00121507 0.00121507 0.00121507
  0.00121507 0.00121507 0.00121507 0.00121507 0.00121507 0.00121507
  0.00121507 0.00121507 0.00121507 0.00121507 0.00121507 0.00121507
  0.00121507 0.00121507 0.00121507 0.00121507 0.00121507 0.00121507
  0.00121507 0.00121507 0.00121507 0.00121507 0.00121507 0.00121507
  0.00121507 0.00121507 0.00121507 0.00121507 0.00121507 0.00121507
  0.00121507 0.00121507 0.00121507 0.00121507 0.00121507 0.00121507
  0.00121507 0.00121507 0.00121507 0.00121507 0.

In [0]:
y = np.dot(x, np.linalg.matrix_power(matrix, 20))
print(y)

[[0.0018454  0.00518145 0.00141799 0.00118602 0.00038477 0.00335973
  0.00056263 0.00041217 0.00040041 0.00082203 0.00122955 0.0004509
  0.00046703 0.00244943 0.00038477 0.00038477 0.00135811 0.00038534
  0.03529431 0.02075895 0.00038477 0.00038477 0.00046863 0.00150989
  0.02092053 0.00049157 0.00070201 0.00082682 0.05511666 0.00357507
  0.00061313 0.00058765 0.00038477 0.00111015 0.01386662 0.00214335
  0.00048481 0.00157168 0.00953573 0.00150956 0.00058779 0.00117392
  0.00120767 0.0004517  0.00047684 0.00057829 0.00229082 0.00038477
  0.0004259  0.00085734 0.00041149 0.00038477 0.00129042 0.0004294
  0.00045615 0.00060219 0.00218425 0.00231401 0.00038477 0.00089076
  0.00065959 0.00039274 0.00250511 0.00111601 0.00060134 0.00161776
  0.00040401 0.0004832  0.00185049 0.00199877 0.00049121 0.00039061
  0.00768328 0.00050581 0.00044208 0.00046997 0.01358123 0.00220213
  0.00038477 0.03764779 0.00078054 0.00145542 0.0006622  0.00042087
  0.00048613 0.00746215 0.00071243 0.00064138 0.00

In [0]:
rating = []
for i, host in enumerate(i2h):
    rating.append((i, y[0][i], host))
rating = sorted(rating, key=lambda x:-x[1])
for i, pr, host in rating:
    print(pr, host)

0.05511665994787871 nytimes.com
0.04232894744295712 bloomberg.com
0.037647787907772755 forbes.com
0.03614312420561435 ria.ru
0.03529431498509583 theguardian.com
0.023043659191782036 reuters.com
0.02092053045639703 mirror.co.uk
0.020758953442467056 bbc.co.uk
0.013871023570674577 kommersant.ru
0.013866615406266254 theverge.com
0.013581232140313225 thesun.co.uk
0.01106746029435335 iz.ru
0.010925324372022406 hollywoodreporter.com
0.010508062691059025 rbc.ru
0.010459412364063347 interfax.ru
0.009666022013528227 russian.rt.com
0.009535732547369119 politico.com
0.00926787785043659 ft.com
0.009175715590080662 telegraph.co.uk
0.0076832763506289635 independent.co.uk
0.007606665179379701 businessinsider.com
0.007462148583606343 nature.com
0.006900816741165872 nypost.com
0.006758676920675725 techcrunch.com
0.006736039250769494 rg.ru
0.00579443048312328 theconversation.com
0.005729029336235546 cbsnews.com
0.0054240465107435925 360tv.ru
0.005181448284523417 foxnews.com
0.004645394774737957 wired.com